<h1 style="color:blue;text-align:center;">Lab4 - Part 1</h1>

<hr style="border:2px solid blue;">

<h3 style="color:blue;">Realised by: <strong style="color:black;">Ouahid Mariyam</strong></h3>
 

<h3 style="color:blue;">Guided by: <strong style="color:black;">Pr . ELAACHAk LOTFI</strong></h3> 
   

<p><strong>Objective :</strong> The main purpose behind this lab is to get familiar with NLP language models
using Pytorch library.</p>


# Summary: Part 1

1. [Part 1: Classification Regression](#part-1)
    1. [Data Collection](#col)
    2. [Data Preprocessing](#pre)
    3. [Model Training](#tra)
    4. [Model Evaluation](#eva)
   
 

# Part 1: Classification Regression <a id='part-1'></a>

## A. Data Collection <a id='col'></a>

We use web scraping libraries such as Scrapy or BeautifulSoup to collect text data from Arabic websites related to a specific topic. Each text is assigned a relevance score between 0 to 10.

In [1]:
# Importing necessary libraries
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import string
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [2]:
# Data Collection
urls = [
    'https://www.who.int/ar/news/item/17-07-1445-who-awards-countries-for-progress-in-eliminating-industrially-produced-trans-fats-for-first-time',
    'https://www.who.int/ar/news/item/17-08-1444-massive-efforts-needed-to-reduce-salt-intake-and-protect-lives',
    'https://www.who.int/ar/news/item/26-12-1444-aspartame-hazard-and-risk-assessment-results-released'
]

In [3]:
def scrape_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    text = ' '.join([p.get_text(strip=True) for p in paragraphs])
    return text

texts = [scrape_url(url) for url in urls]

In [4]:
# Example manual scoring (adjust according to text relevance)
scored_texts = [(texts[0], 6), (texts[1], 7.5)]

# Displaying the collected and scored texts
for i, (text, score) in enumerate(scored_texts):
    print(f'Text {i+1} (Score: {score}):\n{text}\n')


Text 1 (Score: 6):
منحت منظمة الصحة العالمية (المنظمة) خمسة بلدان أولى شهادات على الإطلاق تصادق على إحراز التقدم في التخلّص من الدهون المتحوّلة المنتجة صناعياً. فقد أثبتت كل من الدانمرك وليتوانيا وبولندا والمملكة العربية السعودية وتايلند تنفيذها لسياسات بشأن أفضل الممارسات المتبعة في التخلّص من الدهون المتحوّلة المنتجة صناعياً (الدهون المتحوّلة) مدعومة بما يلزم من نظم لرصد السياسات وإنفاذها. كما أصدرت المنظمة نتائج مستمدة من السنوات الخمس الأولى لتنفيذمبادرتهاREPLACEللتخلّص من الدهون المتحوّلة. ومع أن الغاية الطموحة التي حددتها المنظمة في عام 2018 - للتخلّص تماماً من الدهون المتحوّلة في إمدادات الأغذية العالمية بحلول عام 2023 – هي غاية لم تُبلغ، فقد أُحرِزَ تقدم ملحوظ صوب بلوغها في كل إقليم من أقاليم العالم. ونُفّذت في عام 2023 لوحده سياسات جديدة بشأن أفضل الممارسات في 7 بلدان (هي مصر والمكسيك ومولدوفا ونيجيريا ومقدونيا الشمالية والفلبين وأوكرانيا). والدهون المتحوّلة هي دهون يتراوح قوامها بين شبه الصلب والصلب وتتخذ شكلين اثنين هما: الدهون المنتجة صناعياً وتلك الطبيعية. ويرتبط مدخول الف

## B. Data Preprocessing <a id='pre'></a>

We establish an NLP pipeline for preprocessing the collected dataset. This pipeline includes tokenization, stemming, lemmatization, removal of stop words, and possibly discretization.

In [5]:
# Data Preprocessing
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stemmer = SnowballStemmer('arabic')
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('arabic'))

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in stemmed_tokens]
    return ' '.join(lemmatized_tokens)

# Preprocess the texts
preprocessed_texts = [preprocess_text(text) for text in texts]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# Print the preprocessed texts
for i, text in enumerate(preprocessed_texts):
    print(f'Preprocessed Text {i+1}:\n{text}\n')


Preprocessed Text 1:
منح منظم الصح عالم منظم لدا اولى شهاد اطلاق تصادق احراز تقدم الده منتج فقد اثب دانمرك يتوان ولند مملك عرب سعود تايلند تنفيذ سياس بشء افضل ممارس متبع الده منتج الده مدعوم يلزم نظم لرصد سياس انفاذ اصدر منظم نتايج مستمد سنو خمس اولى الده ومع الغا طموح حدد منظم عام الده امداد اغذ عالم حلول عام غا فقد تقدم ملحوظ صوب لوغ اقليم اقاليم عالم عام لوحد سياس جديد بشء افضل ممارس لدا مصر مكس مولدوف نيجير مقدون شمال فلب اوكران والده دهو يتراوح قوام صلب صلب تتخذ شكل الده منتج وتل طبيع يرتبط مدخول فرد الده زياد خطور اصاب نوب قلب وفا سبب امراض قلب ليس لهذ الده وايد صح تكو اطعم حاو كم كبير اطعم مقل والكع وجب جاهز حاو كم كبير سكر والده ملح يوجد لد سياس معمول بشء افضل ممارس معالج الده شكل كبير بيء غذاء لنح مليار نسب سكا عالم مقارن نسب سنو فقط متوقع تنقذ سياس ارواح شخص تحدث دكتور تيدروس ادحانوم مدير عام الده وايد صح معروف مخاطر الصح جسيم ونح سعداء للغا لان كثير وضع سياس تحظر استعمال الده تحد استعمال اغذ وضع سياس تنفيذ شيء اخر وان اهنء دانمرك يتوان ولند مملك عرب سعود تايلند تتولى قياد عا

## C. Model Training <a id='tra'></a>

We train four different recurrent neural network (RNN) architectures - RNN, Bidirectional RNN, GRU, and LSTM - on the preprocessed dataset. We tune hyperparameters to optimize performance.

In [7]:
# Model Training 
class TextDataset(Dataset):
    def __init__(self, texts, scores):
        self.texts = texts
        self.scores = scores

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.scores[idx]

In [8]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

In [9]:
class BiRNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiRNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

In [10]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

In [11]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [12]:
# Function to train the model
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for texts, scores in dataloader:
            texts = texts.float()
            scores = scores.float().view(-1, 1)

            outputs = model(texts)
            loss = criterion(outputs, scores)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

## D. Model Evaluation<a id='eva'></a>

We evaluate the trained models using standard metrics such as Mean Squared Error (MSE) and Mean Absolute Error (MAE). Additionally, we may consider other metrics like BLEU score for evaluation.

In [13]:
# Function to evaluate the model
def evaluate_model(model, dataloader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for texts, scores in dataloader:
            texts = texts.float()
            scores = scores.float().view(-1, 1)

            outputs = model(texts)
            predictions.extend(outputs.numpy())
            actuals.extend(scores.numpy())

    mse = mean_squared_error(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    return mse, mae # Return the evaluation metrics

In [14]:
# Sample preprocessed text data (vectorized for the sake of example)
preprocessed_texts = [np.random.rand(10, 100) for _ in range(2)]  # Example 2 texts, each with 10 words, 100-dim embeddings
scores = [6, 7.5]

In [15]:
# Splitting the data into training and testing sets
train_texts, test_texts, train_scores, test_scores = train_test_split(preprocessed_texts, scores, test_size=0.2, random_state=42)

# Converting lists of numpy arrays to PyTorch tensors
train_texts = torch.tensor(train_texts, dtype=torch.float32)
test_texts = torch.tensor(test_texts, dtype=torch.float32)
train_scores = torch.tensor(train_scores, dtype=torch.float32)
test_scores = torch.tensor(test_scores, dtype=torch.float32)

# Creating custom dataset instances for training and testing data
train_dataset = TextDataset(train_texts, train_scores)
test_dataset = TextDataset(test_texts, test_scores)

# Creating data loaders for training and testing datasets
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10868\2412606312.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  train_texts = torch.tensor(train_texts, dtype=torch.float32)


In [16]:
# Defining hyperparameters
input_size = 100 
hidden_size = 128
output_size = 1
num_epochs = 10
learning_rate = 0.001

# Initializing the different models to be trained
models = {
    'RNN': RNNModel(input_size, hidden_size, output_size),
    'BiRNN': BiRNNModel(input_size, hidden_size, output_size),
    'GRU': GRUModel(input_size, hidden_size, output_size),
    'LSTM': LSTMModel(input_size, hidden_size, output_size)
}

In [17]:
# Training and evaluating each model
for model_name, model in models.items():
    print(f'Training {model_name} model...')
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_model(model, train_dataloader, criterion, optimizer, num_epochs)

    mse, mae = evaluate_model(model, test_dataloader)
    print(f'{model_name} model Evaluation:')
    print(f' - Mean Squared Error: {mse:.4f}')
    print(f' - Mean Absolute Error: {mae:.4f}')

Training RNN model...
Epoch [1/10], Loss: 37.9811
Epoch [2/10], Loss: 31.4718
Epoch [3/10], Loss: 25.8470
Epoch [4/10], Loss: 21.0031
Epoch [5/10], Loss: 16.8329
Epoch [6/10], Loss: 13.2703
Epoch [7/10], Loss: 10.2764
Epoch [8/10], Loss: 7.8106
Epoch [9/10], Loss: 5.8205
Epoch [10/10], Loss: 4.2441
RNN model Evaluation:
 - Mean Squared Error: 13.7539
 - Mean Absolute Error: 3.7086
Training BiRNN model...
Epoch [1/10], Loss: 37.3675
Epoch [2/10], Loss: 30.4698
Epoch [3/10], Loss: 24.3927
Epoch [4/10], Loss: 19.1087
Epoch [5/10], Loss: 14.5885
Epoch [6/10], Loss: 10.8007
Epoch [7/10], Loss: 7.7029
Epoch [8/10], Loss: 5.2396
Epoch [9/10], Loss: 3.3465
Epoch [10/10], Loss: 1.9548
BiRNN model Evaluation:
 - Mean Squared Error: 9.9278
 - Mean Absolute Error: 3.1508
Training GRU model...
Epoch [1/10], Loss: 32.6675
Epoch [2/10], Loss: 28.4278
Epoch [3/10], Loss: 24.5070
Epoch [4/10], Loss: 20.8555
Epoch [5/10], Loss: 17.4465
Epoch [6/10], Loss: 14.2802
Epoch [7/10], Loss: 11.3831
Epoch [8/10]